In [1]:
from strawberryfields.apps import train

In [2]:
import networkx as nx
from strawberryfields.apps import plot

graph = nx.lollipop_graph(3, 2)
A = nx.to_numpy_array(graph)
plot.graph(graph)

In [3]:
nr_modes = len(A)
weights = train.Exp(nr_modes)

In [4]:
n_mean = 6
vgbs = train.VGBS(A, n_mean, weights, threshold=False)

In [7]:
cost = train.Stochastic(h, vgbs)

In [6]:
import numpy as np

def h(s):
    not_subset = [k for k in range(len(s)) if k not in subset]
    return sum(s[not_subset]) - sum(s[subset])

np.random.seed(1969)  # for reproducibility
d = nr_modes
params = np.zeros(d)
subset = [0, 1, 2]
nr_samples = 100

print('Initial mean photon numbers = ', vgbs.mean_photons_by_mode(params))

Initial mean photon numbers =  [1.3272603  1.3272603  2.0720985  0.94658785 0.32679304]


In [8]:
nr_steps = 100
rate = 0.01

for i in range(nr_steps):
    params -= rate * cost.grad(params, nr_samples)
    if i % 50 == 0:
        print('Cost = {:.3f}'.format(cost.evaluate(params, nr_samples)))

print('Final mean photon numbers = ', vgbs.mean_photons_by_mode(params))

Cost = -3.023
Cost = -3.603
Final mean photon numbers =  [3.02008655 3.10323569 3.5981762  1.1573517  0.26527468]


In [9]:
Aw = vgbs.A(params)
samples = vgbs.generate_samples(Aw, n_samples=10)
print(samples)

[[2 2 0 0 0]
 [0 0 0 0 0]
 [1 1 4 2 0]
 [4 5 5 3 1]
 [0 2 2 0 0]
 [2 0 2 1 1]
 [1 0 1 0 0]
 [1 1 4 2 0]
 [4 5 5 2 0]
 [4 4 4 0 0]]


In [10]:
from strawberryfields.apps import data

data_pl = data.Planted()
data_samples = data_pl[:1000]  # we use only the first one thousand samples
A = data_pl.adj
nr_modes = len(A)

In [11]:
weights = train.Exp(nr_modes)
n_mean = 1
vgbs = train.VGBS(A, n_mean, weights, threshold=True)
cost = train.KL(data_samples, vgbs)

In [12]:
from numpy.linalg import norm

params = np.zeros(nr_modes)
steps = 1000
rate = 0.15

for i in range(steps):
    params -= rate * cost.grad(params)
    if i % 100 == 0:
        diff = cost.mean_n_data - vgbs.mean_clicks_by_mode(params)
        print('Norm of difference = {:.5f}'.format(norm(diff)))

Norm of difference = 0.90856
Norm of difference = 0.01818
Norm of difference = 0.00532
Norm of difference = 0.00177
Norm of difference = 0.00062
Norm of difference = 0.00022
Norm of difference = 0.00008
Norm of difference = 0.00003
Norm of difference = 0.00001
Norm of difference = 0.00000


In [13]:
Aw = vgbs.A(params)
samples = vgbs.generate_samples(Aw, n_samples=10)
print(samples)

[[1 1 0 0 0 1 0 1 1 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1 1 1 1 0 1 1 1 0 1]
 [1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 1 0 0 1 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 1 1 1 0 1 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 1 0 1 1]
 [0 0 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0]]
